<a href="https://colab.research.google.com/github/ludmilaexbrayat/recognizing-packagings/blob/master/generate_y_target.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Code to read csv file into Colaboratory:
#!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
link1='https://drive.google.com/open?id=1Ptvds34C9RaRuoUbb6fAm7qIKhGZY3KQ'
fluff1, id1 = link1.split('=')
print(id1) # Verify that you have everything after '='

1Ptvds34C9RaRuoUbb6fAm7qIKhGZY3KQ


In [0]:
downloaded = drive.CreateFile({'id':id1}) 
downloaded.GetContentFile('en.openfoodfacts.org.products.csv')  

In [30]:
link2='https://drive.google.com/open?id=1v8wUURo6NCl4HxmFSb9zZ17dTbj6STaJ'
fluff2, id2 = link2.split('=')
print(id2) # Verify that you have everything after '='

1v8wUURo6NCl4HxmFSb9zZ17dTbj6STaJ


In [0]:
downloaded = drive.CreateFile({'id':id2}) 
downloaded.GetContentFile('voc_dico.csv')

CYP'S PART

In [0]:
df3 = pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t', encoding='UTF-8')
# Dataset is now stored in a Pandas Dataframe

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,10,11,17,18,19,20,24,25,26,28,29,30,35,37,40,44,45,46,49,52,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df3.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,...,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,NaN,NaN,courte paille,courte-paille,"Epicerie,Condiments,Sauces,Moutardes","en:groceries,en:condiments,en:sauces,en:mustards","Groceries,Condiments,Sauces,Mustards",NaN,NaN,NaN,NaN,Delois france,fr:delois-france,fr:delois-france,NaN,NaN,NaN,NaN,NaN,NaN,courte paille,France,en:france,France,eau graines de téguments de moutarde vinaigre ...,moutarde,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,NaN,NaN,Watt,watt,fr:Xsf,fr:xsf,fr:xsf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
df3.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name',
       'generic_name', 'quantity',
       ...
       'carbon-footprint-from-meat-or-fish_100g', 'nutrition-score-fr_100g',
       'nutrition-score-uk_100g', 'glycemic-index_100g', 'water-hardness_100g',
       'choline_100g', 'phylloquinone_100g', 'beta-glucan_100g',
       'inositol_100g', 'carnitine_100g'],
      dtype='object', length=175)

Create the y table (target)

In [0]:
df = df3[['code', 'countries','packaging_tags']]

In [0]:
df.head()

,code,countries,packaging_tags
0,0000000000017,France,NaN
1,0000000000031,France,NaN
2,0000000000100,France,NaN
3,00000000001111111111,en:France,NaN
4,0000000000123,France,NaN


In [0]:
df.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
df.shape

(185048, 3)

In [0]:
df.countries.nunique()

4072

In [0]:
df = df[df['countries'].str.contains("France")]

In [0]:
df.countries.unique()

array(['France', 'France, États-Unis', 'France,Switzerland', ...,
       'Australie, Belgique, France, Japon, Nouvelle-Zélande, Royaume-Uni, États-Unis',
       'Australia,Belgium,France,Japan,New Zealand,United Kingdom',
       'France, Malaysia'], dtype=object)

In [0]:
df.packaging_tags.value_counts()[:50]

sachet,plastique               8559
plastique                      4800
bouteille,verre                3637
carton                         3384
barquette,plastique            2914
sachet                         2634
conserve                       2552
bocal,verre                    2460
frais,plastique                2290
bouteille,plastique            2235
carton,plastique               2032
frais                          1927
bouteille                      1599
sachet-plastique               1531
barquette                      1224
boite                          1119
conserve,metal                 1000
bocal,verre,couvercle,metal     927
brique,carton                   915
sachet,plastique,surgele        879
carton,boite,plastique          870
pot                             870
carton,surgele                  795
boite,carton                    789
bocal                           764
verre                           763
barquette,plastique,frais       759
bocal,verre,metal           

In [0]:
packages_to_keep = ['plastique', 'conserve', 'metal', 'carton', 'verre', 'bocal', 'brique', 'pot-en-verre', 'papier', 'bouteille-plastique', 'bouteille-en-verre', 'bouteille-verre']
len(packages_to_keep)

12

In [0]:
df_clean = df.copy()
df_clean['packaging_tags'] = df.packaging_tags.str.strip().str.split(',')

In [0]:
df_clean.head()

,code,countries,packaging_tags
16,0000000001281,France,"[barquette, plastique]"
140,0000000020114,France,"[bocal, plastique]"
144,0000000024600,France,[plastique]
296,0000000274722,France,"[carton, plastique]"
297,0000000274739,France,"[plastique, carton]"


In [0]:
df_clean['packaging_tags_reduced'] = df_clean['packaging_tags'].apply(lambda l: list(set(l) & set(packages_to_keep)))

In [0]:
df_clean.head()

,code,countries,packaging_tags,packaging_tags_reduced
16,0000000001281,France,"[barquette, plastique]",[plastique]
140,0000000020114,France,"[bocal, plastique]","[bocal, plastique]"
144,0000000024600,France,[plastique],[plastique]
296,0000000274722,France,"[carton, plastique]","[plastique, carton]"
297,0000000274739,France,"[plastique, carton]","[plastique, carton]"


In [0]:
df_clean.shape

(124524, 4)

In [0]:
y0 = df_clean[df_clean['packaging_tags_reduced'].map(lambda l: len(l) > 0)]

In [0]:
y0.shape

(88685, 4)

One-hot encoding of the packaging_tags_reduced column

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
mlb = MultiLabelBinarizer(classes=packages_to_keep)

In [0]:
one_hot_encoding = pd.DataFrame(mlb.fit_transform(y0['packaging_tags_reduced']),columns=mlb.classes_, index=y0.index)
one_hot_encoding.head()

,plastique,conserve,metal,carton,verre,bocal,brique,pot-en-verre,papier,bouteille-plastique,bouteille-en-verre,bouteille-verre
16,1,0,0,0,0,0,0,0,0,0,0,0
140,1,0,0,0,0,1,0,0,0,0,0,0
144,1,0,0,0,0,0,0,0,0,0,0,0
296,1,0,0,1,0,0,0,0,0,0,0,0
297,1,0,0,1,0,0,0,0,0,0,0,0


In [0]:
y1 = pd.concat([y0['code'], one_hot_encoding], axis=1, sort=False)

In [0]:
y1.reset_index(drop=True, inplace=True)

In [0]:
y1.head()

,code,plastique,conserve,metal,carton,verre,bocal,brique,pot-en-verre,papier,bouteille-plastique,bouteille-en-verre,bouteille-verre
0,0000000001281,1,0,0,0,0,0,0,0,0,0,0,0
1,0000000020114,1,0,0,0,0,1,0,0,0,0,0,0
2,0000000024600,1,0,0,0,0,0,0,0,0,0,0,0
3,0000000274722,1,0,0,1,0,0,0,0,0,0,0,0
4,0000000274739,1,0,0,1,0,0,0,0,0,0,0,0


In [0]:
y1.shape

(88685, 13)

TRISTAN AND MATHIEU'S PART

In [32]:
df = pd.read_csv('en.openfoodfacts.org.products.csv', delimiter='\t', encoding='UTF-8')
df_dic = pd.read_csv('voc_dico.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,10,11,17,18,19,20,24,25,26,28,29,30,35,37,40,44,45,46,49,52,66,67) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
#clean df
df = df[~df['image_url'].isna()]
df= df[['code', 'packaging_tags']].dropna().drop_duplicates('code')
df.head()

,code,packaging_tags
16,0000000001281,"barquette,plastique"
140,0000000020114,"bocal,plastique"
144,0000000024600,plastique
296,0000000274722,"carton,plastique"
297,0000000274739,"plastique,carton"


In [34]:
df_dic.head()

,Label,Col,Col1
0,plastique,Plastique,NaN
1,plastiktute,Plastique,NaN
2,2,NaN,NaN
3,4-pe-ld,NaN,NaN
4,fr-bouteille-verre,Verre,Bouteille


In [0]:
#get dictionaries to replace values
dic_transfo1 = dict(zip(df_dic.Label, df_dic.Col))
dic_transfo2 = dict(zip(df_dic.Label, df_dic.Col1))

In [36]:
df_1 = df.set_index('code')['packaging_tags'].str.split(',', expand=True)
value_vars_1 = df_1.columns 
df_1 = df_1.reset_index()
df_2 = pd.melt(df_1, id_vars = 'code', value_vars = value_vars_1).dropna()[['code','value']]
df_2.head()

,code,value
0,0000000001281,barquette
1,0000000020114,bocal
2,0000000024600,plastique
3,0000000274722,carton
4,0000000274739,plastique


In [37]:
df_2['brico 1'] = df_2['value'].map(dic_transfo1)
df_2['brico 2'] = df_2['value'].map(dic_transfo2)
df_2.head()

,code,value,brico 1,brico 2
0,0000000001281,barquette,NaN,Barquette
1,0000000020114,bocal,NaN,Bocal
2,0000000024600,plastique,Plastique,NaN
3,0000000274722,carton,Carton,NaN
4,0000000274739,plastique,Plastique,NaN


In [0]:
df_3 = pd.melt(df_2, id_vars='code', value_vars = ['brico 1', 'brico 2']).dropna()[['code','value']]
df_4 = pd.get_dummies(df_3.set_index('code')['value']).reset_index().groupby('code').sum()

In [44]:
df_4.head()

,Barquette,Bocal,Boite,Bouteille,Brique,Carton,Conserve,Metal,Papier,Plastique,Pot,Sachet,Verre
code,,,,,,,,,,,,,
85,0,0,0,0,0,0,0,0,0,1,0,0,0
133,0,0,0,0,0,0,0,0,0,0,0,0,1
1151,1,0,1,0,0,1,0,0,0,1,0,0,0
1202,0,0,0,0,0,0,0,0,0,1,0,1,0
1426,0,0,0,0,0,0,0,1,0,0,0,0,0
